In [9]:
import pandas as pd
from datetime import datetime

from os.path import join

from pprint import pprint

from typing import List

In [ ]:
RAW_FOLDER_PATH = "./../data/raw"
CLEANED_FOLDER_PATH = "./../data/cleaned"

file_name = "Swiggy 09-04-2025 - 22-06-2025.xlsx"
file_name = "Swiggy 01-01-2025 - 15-04-2025.xlsx"
file_name = "Swiggy 22-06-2025 - 10-07-2025.xlsx"
file_name = "ZEPTO.xlsx"
file_name = "Zepto 10-07-2025 - 20-06-2025.xlsx"

excelBook: dict[str, pd.DataFrame] = pd.read_excel(
    join(RAW_FOLDER_PATH, file_name), sheet_name=None
)

In [52]:
class Extract:
    def __checkColumnName(self, col: pd.Series):
        return (
            col.astype(str)
            .apply(
                lambda string: " ".join(
                    list(
                        filter(
                            lambda x: not x.isspace() and x,
                            string.lower().replace("\n", "").split(" "),
                        )
                    )
                )
            )
            .isin(list(map(lambda x: x.lower(), self.columns)))
            .any()
        )

    def __init__(self, column: List[str]):
        self.columns: List[str] = column

        self.processedDataFrame: pd.DataFrame = pd.DataFrame(columns=self.columns)

    def extractTable(self, row: pd.Series) -> bool:
        return str(row[self.columns[0]]).isnumeric()

    def filterColumns(self, df: pd.DataFrame) -> List[str]:
        columns = []
        return columns

    def extractDate(self, key):
        dateFormat = "%d-%m-%Y"
        try:
            return datetime.strptime(key[:10], dateFormat)
        except:
            return False

    def extractLocation(self, key: str):
        return key[10:].strip().upper()

    def extractDataFrame(self, sheetName: str, sheetData: pd.DataFrame):
        # sheetData.columns = self.columns[:-2]
        date = self.extractDate(sheetName)
        if date == False:
            return pd.DataFrame()

        sheetData = sheetData[
            sheetData.columns[sheetData.apply(self.__checkColumnName, axis=0)]
        ].copy()

        # print(sheetData.columns, sheetName, sheetData.head(10))
        sheetData.columns = self.columns
        sheetData = sheetData[sheetData.apply(self.extractTable, axis=1)].copy()

        sheetData["Date"] = date
        sheetData["Location"] = self.extractLocation(sheetName)

        return sheetData

    def add(self, dataFrame: pd.DataFrame):
        dataFrame = dataFrame.dropna(how="all", axis=1)
        if not dataFrame.empty:
            self.processedDataFrame = pd.concat(
                [self.processedDataFrame, dataFrame], ignore_index=True
            )

In [55]:
extract = Extract(
    [
        "No",
        # "Article Code",
        "Article Name",
        # "Item Description",
        # "Uom",
        "UoM",
        # "Dispatched Qty",
        "Invoice Qty.",
        # "Received Qty",
        "Rate",
        # "Total Amount",
        "Amount",
        # "Date",
        # "Location",
    ]
)


errorSheets: List[str] = []


for sheetName, sheetData in excelBook.items():
    if sheetData.empty:
        errorSheets.append(sheetName)
        continue

    try:
        df = extract.extractDataFrame(sheetName, sheetData)
        # pass
    except:
        # print(sheetData, sheetData.columns.__len__())
        errorSheets.append(sheetName)
        break

    if not df.empty:
        extract.add(df)
    else:
        errorSheets.append(sheetName)



print("The following sheets are not parsed: ")
pprint(errorSheets)

The following sheets are not parsed: 
[]


In [56]:
extract.processedDataFrame.reset_index(drop=True).to_csv(join(CLEANED_FOLDER_PATH, file_name.rsplit(".", 1)[0]) + ".csv")

In [2]:
import re